In this notebook, Bayesian Optimisation is used on a fundamentally 1D problem, pretending to be 2D with the addition of a nuisance parameter.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import sklearn.gaussian_process as gp
import matplotlib.pyplot as plt
import seaborn as sns # prettify matplotlib

In [ ]:
# local modules
import sys; sys.path.append('..')
import optimisation as op
from optimisation import plot3D
from utils import synthetic_data

In [ ]:
# Make deterministic
np.random.seed(100)

# Target Function

In [ ]:
xs = np.linspace(-2, 10, 100)
nuisance = np.linspace(0, 10, 100)

noise = synthetic_data.Noise1D(xs, 0.0)
f = lambda x: np.exp(-(x - 2)**2) + np.exp(-(x - 6)**2/10) + 1/ (x**2 + 1) + noise.get(x)

# testing log function with nuisance parameter
#xs = op.logspace(0.005, 5, num_per_mag=200)
#f = lambda x: np.cos(2*(20-x)**2)/x - 2*np.log(x)

f2D = lambda x, nuisance: f(x)
ys = f(xs)
best_y = np.max(ys)

X,Y = np.meshgrid(xs, nuisance)
Z = f2D(X, Y)

In [ ]:
plt.figure(figsize=(16,6))
plt.plot(xs, ys)
plt.show()

In [ ]:
plot3D.surface3D(X, Y, Z)

In [ ]:
ranges1D = {'x' : xs}
ranges2D = {'x' : xs, 'nuisance' : nuisance}

class TestEvaluator(op.Evaluator):
    def test_config(self, config):
        return f(config.x)
evaluator = TestEvaluator()

gp_params = {
    'alpha': 1e-10, # default noise level, deal with noise with the white kernel
    'kernel': 1.0 * gp.kernels.Matern(nu=1.5) + gp.kernels.WhiteKernel(),
    'n_restarts_optimizer': 4
}
strategy = op.AcquisitionStrategy(
    pre_phase_steps = 4,
    acquisition_function = ('UCB', {'kappa' : 5})
)

def create_optimiser(ranges):
    return op.BayesianOptimisationOptimiser(
        ranges, maximise_cost=True, acquisition_strategy=strategy,
        gp_params=gp_params, maximisation_args=None, close_tolerance=1e-4
    )
bo1D = create_optimiser(ranges1D)
bo2D = create_optimiser(ranges2D)

In [ ]:
op.gui.optimiser_progress_bar(bo1D)
bo1D.run_sequential(evaluator, max_jobs=25)

In [ ]:
bo1D.plot_cost_over_time(true_best=best_y);

In [ ]:
def plot(n, step):
    bo1D.plot_step_1D('x', n, true_cost=f);
op.gui.step_log_slider(bo1D, plot, pre_compute=False);

In [ ]:
op.gui.optimiser_progress_bar(bo2D)
bo2D.run_sequential(evaluator, max_jobs=60)

In [ ]:
bo2D.plot_cost_over_time(true_best=best_y);

In [ ]:
def plot(n, step):
    bo2D.plot_step_1D('x', n, true_cost=f);
op.gui.step_log_slider(bo2D, plot, pre_compute=False);

In [ ]:
def plot(n, step):
    bo2D.plot_step_1D('nuisance', n);
op.gui.step_log_slider(bo2D, plot, pre_compute=False);

In [ ]:
def plot2D(n, step):
    bo2D.plot_step_2D('x', 'nuisance', n, true_cost=Z, force_view_linear=False)
op.gui.step_log_slider(bo2D, plot2D, pre_compute=False);

In [ ]:
bo2D.scatter_plot('x', 'nuisance')

In [ ]:
ra1D = op.RandomSearchOptimiser(ranges1D, maximise_cost=True)
ra2D = op.RandomSearchOptimiser(ranges2D, maximise_cost=True)

In [ ]:
ra1D.run_sequential(evaluator, max_jobs=60)

In [ ]:
ra1D.plot_cost_over_time(true_best=best_y);

In [ ]:
ra2D.run_sequential(evaluator, max_jobs=60)

In [ ]:
ra2D.plot_cost_over_time(true_best=best_y);

In [ ]:
ra2D.scatter_plot('x', 'nuisance')